## Laboratorio 5 - Autocodificadores

In [211]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, Model

# 1.1 Cargar los datos
df = pd.read_csv('UK_foods.csv', index_col=0)  # Suponiendo que la primera columna contiene los nombres de los alimentos

# Transponer el DataFrame para que los países sean las observaciones y los alimentos las características
df = df.transpose()

print(df.head())

           Cheese  Carcass_meat   Other_meat   Fish  Fats_and_oils   Sugars  \
England       105            245          685   147             193     156   
Wales         103            227          803   160             235     175   
Scotland      103            242          750   122             184     147   
N.Ireland      66            267          586    93             209     139   

           Fresh_potatoes   Fresh_Veg   Other_Veg   Processed_potatoes   \
England                720         253         488                  198   
Wales                  874         265         570                  203   
Scotland               566         171         418                  220   
N.Ireland             1033         143         355                  187   

           Processed_Veg   Fresh_fruit   Cereals   Beverages  Soft_drinks   \
England               360          1102      1472         57          1374   
Wales                 365          1137      1582         73          12

In [212]:
# 1.2 Exploración básica
print(df.describe())

           Cheese  Carcass_meat   Other_meat         Fish  Fats_and_oils   \
count    4.000000           4.00     4.000000    4.000000        4.000000   
mean    94.250000         245.25   706.000000  130.500000      205.250000   
std     18.856917          16.50    93.427334   29.557853       22.366269   
min     66.000000         227.00   586.000000   93.000000      184.000000   
25%     93.750000         238.25   660.250000  114.750000      190.750000   
50%    103.000000         243.50   717.500000  134.500000      201.000000   
75%    103.500000         250.50   763.250000  150.250000      215.500000   
max    105.000000         267.00   803.000000  160.000000      235.000000   

          Sugars  Fresh_potatoes   Fresh_Veg   Other_Veg   \
count    4.00000         4.000000    4.000000    4.000000   
mean   154.25000       798.250000  208.000000  457.750000   
std     15.47848       200.755863   60.188592   92.471167   
min    139.00000       566.000000  143.000000  355.000000   
2

In [213]:
# 1.3 Normalización
scaler = StandardScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df), columns=df.columns, index=df.index)
print(df_normalized)

             Cheese  Carcass_meat   Other_meat       Fish  Fats_and_oils   \
England    0.658275      -0.017495    -0.259546  0.644585       -0.632429   
Wales      0.535805      -1.277169     1.198856  1.152440        1.535899   
Scotland   0.535805      -0.227441     0.543811 -0.332059       -1.097071   
N.Ireland -1.729885       1.522105    -1.483121 -1.464967        0.193601   

             Sugars  Fresh_potatoes   Fresh_Veg   Other_Veg   \
England    0.130551        -0.450076    0.863312    0.377736   
Wales      1.547958         0.435696    1.093528    1.401682   
Scotland  -0.540853        -1.335847   -0.709834   -0.496364   
N.Ireland -1.137656         1.350227   -1.247006   -1.283054   

           Processed_potatoes   Processed_Veg   Fresh_fruit   Cereals   \
England              -0.336265        0.805477      0.737407 -0.644322   
Wales                 0.084066        1.171603      0.929297  1.679463   
Scotland              1.513193       -0.878702     -0.057567 -0.855575 

In [214]:
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
import numpy as np

# 2.1 Usar todas las observaciones para entrenar
X_train = df_normalized.values

# Número de características o dimensiones
num_features = X_train.shape[1]

# 2.2 Definir la arquitectura
input_layer = layers.Input(shape=(num_features,))
encoder = layers.Dense(4, activation='relu')(input_layer)
bottleneck = layers.Dense(2, activation='relu')(encoder)
decoder = layers.Dense(4, activation='relu')(bottleneck)
output_layer = layers.Dense(num_features, activation='sigmoid')(decoder)

autoencoder = Model(inputs=input_layer, outputs=output_layer)

# Modelo para la reducción de dimensionalidad
encoder_model = Model(inputs=input_layer, outputs=bottleneck)

# 2.3 Compilación y entrenamiento
autoencoder.compile(loss="mse", optimizer=Adam(lr=0.001))
autoencoder.fit(X_train, X_train, epochs=100, batch_size=1)

# 2.4 Evaluación
loss = autoencoder.evaluate(X_train, X_train)
print(f"Loss: {loss}")


Epoch 1/100
4/4 [==============================] - 1s 7ms/step - loss: 1.2526
Epoch 2/100
4/4 [==============================] - 0s 4ms/step - loss: 1.2516
Epoch 3/100
4/4 [==============================] - 0s 4ms/step - loss: 1.2507
Epoch 4/100
4/4 [==============================] - 0s 0s/step - loss: 1.2498
Epoch 5/100
4/4 [==============================] - 0s 5ms/step - loss: 1.2491
Epoch 6/100
4/4 [==============================] - 0s 5ms/step - loss: 1.2483
Epoch 7/100
4/4 [==============================] - 0s 0s/step - loss: 1.2474
Epoch 8/100
4/4 [==============================] - 0s 0s/step - loss: 1.2468
Epoch 9/100
4/4 [==============================] - 0s 5ms/step - loss: 1.2464
Epoch 10/100
4/4 [==============================] - 0s 0s/step - loss: 1.2461
Epoch 11/100
4/4 [==============================] - 0s 0s/step - loss: 1.2455
Epoch 12/100
4/4 [==============================] - 0s 839us/step - loss: 1.2452
Epoch 13/100
4/4 [==============================] - 0s 5ms/step 

In [215]:
# 3.1 Use the autoencoder to reduce the dimensions of the dataset to 2.
encoded_data = encoder_model.predict(df_normalized)

print(encoded_data.shape)  # Debería mostrar (17, 2) si todo fue correcto

1/1 [==============================] - 0s 46ms/step
(4, 2)


In [216]:
# Setear una semilla para evitar mucha variacion entre los datos 
np.random.seed(0)
tf.random.set_seed(0)

# Asegurando que los datos codificados tienen la forma correcta
assert encoded_data.shape == (4, 2), "La forma de los datos codificados no es correcta."

# Creando un dataframe para las dimensiones codificadas
df_encoded = pd.DataFrame(encoded_data, columns=['Dimension 1', 'Dimension 2'])
df_encoded['Country'] = df_normalized.index  # Usar las filas del DataFrame normalizado como 'Country'

# Dibujo del gráfico
fig = px.scatter(df_encoded, 
                 x="Dimension 1", 
                 y="Dimension 2", 
                 color="Country",
                 title="Representación 2D de los datos codificados por país",
                 width=800,
                 height=600)

fig.show()
